In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
naver_df =  pd.read_csv('일단_result2.csv', index_col=0)

In [3]:
naver_df = naver_df.reset_index(drop=True)

In [4]:
naver_df

,movieNm,openDt,total_money,audience,screen_num,show_num,배급사,movieCd,director,prdtYear,typeNm,repNationNm,repGenreNm,showTm,watchGradeNm,actor_1,actor_2,actor_3,companyNm
0,택시운전사,20170802.0,95853645649,12186327,1906,184075,(주)쇼박스,20162869,장훈,2017.0,장편,한국,드라마,137.0,15세이상관람가,이수민,송강호,토마스 크레취만,더램프(주)
1,신과함께-죄와 벌,20171220.0,68350496432,8539495,1912,91301,롯데쇼핑㈜롯데엔터테인먼트,20150976,김용화,2017.0,장편,한국,판타지,139.0,12세이상관람가,하정우,차태현,주지훈,리얼라이즈픽쳐스(주)
2,공조,20170118.0,63783051326,7817618,1392,142989,(주)씨제이이엔엠,20151228,김성훈,2016.0,장편,한국,액션,125.0,15세이상관람가,이재남,현빈,유해진,(주)제이케이필름
3,스파이더맨: 홈 커밍,20170705.0,59125813820,7258678,1965,164448,소니픽쳐스엔터테인먼트코리아주식회사극장배급지점,20167303,존 왓츠,2017.0,장편,미국,액션,133.0,12세이상관람가,제이콥 배덜런,톰 홀랜드,로버트 다우니 주니어,소니픽쳐스엔터테인먼트코리아주식회사극장배급지점
4,범죄도시,20171003.0,56318014149,6879841,1315,149334,"메가박스중앙(주)플러스엠,(주)키위미디어그룹",20172742,강윤성,2017.0,장편,한국,범죄,121.0,청소년관람불가,마동석,윤계상,조재윤,(주)홍필름
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4640,소라닌,20100826.0,81223000,10913,13.0,578.0,(주)스폰지이엔티,20100775,미키 타카히로,2010.0,장편,일본,드라마,125.0,12세이상관람가,코라 켄고,미야자키 아오이,NaN,아스믹에이스엔터테인먼트
4641,춤추는 꿈틀이 밴드,20100204.0,72851000,10885,47.0,718.0,에이원엔터테인먼트,20100011,토마스 보르히 닐슨,2008.0,장편,덴마크,애니메이션,80.0,전체관람가,엄상현,이장원,조현정,디스코 오르메네
4642,내 남자의 순이,20100527.0,73108500,10449,99.0,2013.0,(주)새인컴퍼니,20100277,김호준,2010.0,장편,한국,코미디,91.0,15세이상관람가,박해미,이태성,신이,여해 엔터테인먼트
4643,샤인 어 라이트(1st 지드래곤 콘서트),20100415.0,91164000,10349,14.0,227.0,(주)와이지엔터테인먼트,20100201,정치영,2010.0,장편,한국,다큐멘터리,101.0,12세이상관람가,NaN,NaN,NaN,(주)와이지엔터테인먼트


In [4]:
naver_df_nm = naver_df['movieNm']

In [5]:
naver_df_nm

0                        택시운전사
1                    신과함께-죄와 벌
2                           공조
3                  스파이더맨: 홈 커밍
4                         범죄도시
                 ...          
4640                       소라닌
4641                춤추는 꿈틀이 밴드
4642                  내 남자의 순이
4643    샤인 어 라이트(1st 지드래곤 콘서트)
4644                    셉템버 이슈
Name: movieNm, Length: 4645, dtype: object

In [6]:
def get_star_score_url(naver_df):
    url_df = pd.DataFrame(columns=["movieCd","movieNm_x","movieNm_y","url"])
    
    for n, i in enumerate(list(naver_df_nm)):
        try:
            data = ("http://movie.naver.com/movie/search/result.naver?query={movieNm}&section=all&ie=utf8").format(movieNm=i)
            r = requests.get(data)
            dom = BeautifulSoup(r.content, "html.parser")
            dom1 = dom.select("p.result_thumb")
            open_year = naver_df["prdtYear"].apply(lambda x: str(x)[0:4])
            if dom1[-1] == dom1[0]:
                for i, data in enumerate(dom1):
                    url = data.select_one("a")["href"]
                    movieNm = dom.select("dt")[i].text
                    url_df.loc[len(url_df)] = [
                    naver_df["movieCd"][n],
                    naver_df["movieNm"][n],
                    movieNm,
                    url,]
            elif dom1[-1] != dom1[0]:
                num = 0
                for j in range(0,len(dom.select("dd.etc")),2):
                    year = dom.select("dd.etc")[j].text[-4::]
                    if (year == open_year[n]) & (num == j):
                        num += 10
                        url = dom1[int(j/2)].select_one("a")["href"]
                        movieNm = dom.select("dt")[int(j/2)].text
                        url_df.loc[len(url_df)] = [
                        naver_df["movieCd"][n],
                        naver_df["movieNm"][n],
                        movieNm,
                        url,]
                    num += 2
        except:
            print(i)
    return url_df

In [7]:
url_df = get_star_score_url(naver_df)

반지의 제왕 : 반지원정대 (확장판)
반지의 제왕 : 왕의 귀환 (확장판)
반지의 제왕 : 두개의 탑 (확장판)
아마데우스 감독판
써드 퍼슨 - 감독판
경계의 저편 : I'LL BE HERE -미래편-
경계의 저편 : I'LL BE HERE -과거편-
10,000km(만 킬로미터)
피케이: 별에서 온 얼간이 감독판
일 트로바토레 잘츠부르크 페스티벌
프라이비트 레슨:개인교수
스팔타카스
루키노 비스콘티 회고전
터미네이터2 3D
베드(감독판)
그랑블루 리마스터링 감독판
착한여자 (무삭제)
미조-못다한이야기(감독판)
황비홍의서역웅사
가자! 장미여관으로 3 - 방황 무삭제 감독판
정사: 위험한 성적 유희 무삭제판
남편만 몰라요 (무삭제)
나는 야한 여자가 좋다 2 무삭제감독판
나는 야한 여자가 좋다 무삭제감독판
울언니: 성상납의 비밀 무삭제판
아웃캐스트: 절명도망 무삭제판
명기(무삭제)
따라지: 비열한 거리 무삭제판
포장마차-무삭제판
러브앤서울 복숭아마을 도색소동기 무삭제판
동창회의 목적(무삭제)
뽕2014(무삭제)
외출 무삭제
하숙집 무삭제
어우동: 주인 없는 꽃 무삭제판
광란의 섹스파티 무삭제판
태양을 쏴라 무삭제판
미궁: 비밀애 무삭제 특별판
흔들리는 떡국버스 무삭제판
나는 야한 여자가 좋다 3 무삭제감독판
사쿠란-관능의 시대극 무삭제판
성난 화가 - 무삭제판
루팡 3세 무삭제판
킹 오브 프리즘 (바이 프리티 리듬)
하이 스피드! –프리! 스타팅 데이즈-
비틀스: 에잇 데이즈 어 위크 – 투어링 이어즈
킹 오브 프리즘 –샤이니 세븐 스타즈- IV 루이x신x언노운
비정한 도시 감독판
헨젤과 그레텔: 마녀 사냥꾼 3D
국가대표 완결판-못 다한 이야기
아더와 미니모이-제1탄비밀원정대의출정
2008 서태지 심포니 위드 톨가 카쉬프 로열필하모닉


In [8]:
url_df

,movieCd,movieNm_x,movieNm_y,url
0,20162869,택시운전사,택시운전사 (A Taxi Driver),/movie/bi/mi/basic.naver?code=146469
1,20150976,신과함께-죄와 벌,신과함께-죄와 벌 (Along With the Gods: The Two Worlds),/movie/bi/mi/basic.naver?code=85579
2,20151228,공조,공조 (共助),/movie/bi/mi/basic.naver?code=142384
3,20167303,스파이더맨: 홈 커밍,스파이더맨: 홈커밍 (Spider-Man: Homecoming),/movie/bi/mi/basic.naver?code=135874
4,20172742,범죄도시,범죄도시 (THE OUTLAWS),/movie/bi/mi/basic.naver?code=161242
...,...,...,...,...
4339,20100775,소라닌,소라닌 (ソラニン),/movie/bi/mi/basic.naver?code=52996
4340,20100011,춤추는 꿈틀이 밴드,춤추는 꿈틀이 밴드 (Disco Ormene),/movie/bi/mi/basic.naver?code=73111
4341,20100277,내 남자의 순이,내 남자의 순이,/movie/bi/mi/basic.naver?code=53089
4342,20100201,샤인 어 라이트(1st 지드래곤 콘서트),2009 G-DRAGON 콘서트 SHINE A LIGHT (G-DRAGON LIVE...,/movie/bi/mi/basic.naver?code=74865


In [11]:
url =  pd.read_csv('일단url.csv', index_col=0)

In [24]:
url

,movieCd,movieNm_x,movieNm_y,url
0,20162869,택시운전사,택시운전사 (A Taxi Driver),/movie/bi/mi/basic.naver?code=146469
1,20150976,신과함께-죄와 벌,신과함께-죄와 벌 (Along With the Gods: The Two Worlds),/movie/bi/mi/basic.naver?code=85579
2,20151228,공조,공조 (共助),/movie/bi/mi/basic.naver?code=142384
3,20167303,스파이더맨: 홈 커밍,스파이더맨: 홈커밍 (Spider-Man: Homecoming),/movie/bi/mi/basic.naver?code=135874
4,20172742,범죄도시,범죄도시 (THE OUTLAWS),/movie/bi/mi/basic.naver?code=161242
...,...,...,...,...
4339,20100775,소라닌,소라닌 (ソラニン),/movie/bi/mi/basic.naver?code=52996
4340,20100011,춤추는 꿈틀이 밴드,춤추는 꿈틀이 밴드 (Disco Ormene),/movie/bi/mi/basic.naver?code=73111
4341,20100277,내 남자의 순이,내 남자의 순이,/movie/bi/mi/basic.naver?code=53089
4342,20100201,샤인 어 라이트(1st 지드래곤 콘서트),2009 G-DRAGON 콘서트 SHINE A LIGHT (G-DRAGON LIVE...,/movie/bi/mi/basic.naver?code=74865


In [25]:
url_df["code"] = url_df["url"].apply(lambda x:np.int(x.split("=")[1]))

<ipython-input-25-18d6ed8bb68f>:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  url_df["code"] = url_df["url"].apply(lambda x:np.int(x.split("=")[1]))


In [26]:
url_df["code"]

0       146469
1        85579
2       142384
3       135874
4       161242
         ...  
4339     52996
4340     73111
4341     53089
4342     74865
4343     70463
Name: code, Length: 4344, dtype: int64

평점 -------------------------

In [27]:
def get_star_score(url_df):
    star_score_df = pd.DataFrame(columns=["movieCd","movieNm","star_score","star_user_count","netizen_score","netizen_count"])#,"scene_score","scene_count"])
    for n, i in enumerate(list(url_df["code"])):
        try:
            data = ("http://movie.naver.com/movie/bi/mi/point.nhn?code={code}").format(code=i)
            r = requests.get(data)
            dom = BeautifulSoup(r.content, "html.parser")
            dom1 = dom.select_one("#actualPointPersentBasic")
            dom2 = dom1.select("em")[:]
            dom3 = dom.select_one("#actualPointCountBasic")
            dom4 = dom3.select("em")[0]
            dom5 = dom.select_one("#pointNetizenPersentBasic")
            dom6 = dom5.select("em")[:]
            dom7 = dom.select_one("#pointNetizenCountBasic")
            dom8 = dom7.select("em")[0]
            star_score = ("").join([i.text for i in dom2 [:-1]])
            star_user_count = dom4.text
            netizen_score = ("").join([i.text for i in dom6 [:-1]])
            netizen_count = dom8.text
            star_score_df.loc[len(star_score_df)] = [
                url_df["movieCd"][n],
                url_df["movieNm_x"][n],
                star_score,
                star_user_count,
                netizen_score,
                netizen_count
            ]
        except:
            print(i)
    return star_score_df

In [28]:
star_score_df = get_star_score(url_df)

46758
183288
197495
139835
36962
114233
170469
174460
186675
161355
131410
179713
179713
144089
165470
136131
63853
69325
187582
163878
182644
181033
135970
192324
190611
30215
26407
135011
189938
189938
176513
176513
190615
12162
135606
29870
133047
141477
126745
121081
119472
113537
133076
10235
141959
145420
143304
78060
138071
143312
141416
133056
63516
143275
113560
143265
143259
50054
145340
144122
12847
143230
36916
201852
126318
79448
37748
103328
13213
102999
120047
201865
129714
135602
106364
19501
98103
19529
134740
46580
30857
197467
14964
27844
32613
32614
51368
133091
145398
139712
138261
140126
140111
141488
141422
141471
141410
143350
135614
136029
133290
133270
133266
108070
145368
18353
125742
133165
133179
133244
133227
133245
133189
133166
133523
133237
133151
133163
133140
133152
133154
133153
120547
137415
133138
133139
133226
119413
105902
114244
140106
140105
140104
140103
140101
141494
141753
141498
135427
143325
143329
101427
133514
113215
100106
135351
134904

In [38]:
star_score_df

,movieCd,movieNm,star_score,star_user_count,netizen_score,netizen_count
0,20162869,택시운전사,090.020,"18,929",9.0,"49,617"
1,20150976,신과함께-죄와 벌,080.070,"26,302",7.8,"57,870"
2,20151228,공조,080.080,"14,090",8.2,"25,212"
3,20167303,스파이더맨: 홈 커밍,080.080,"9,933",8.3,"18,792"
4,20172742,범죄도시,090.020,"11,928",9.1,"29,016"
...,...,...,...,...,...,...
3879,20101283,나루토 질풍전 극장판 4 - 더 로스트 타워,0,0,8.1,216
3880,20100775,소라닌,0,0,8.4,335
3881,20100011,춤추는 꿈틀이 밴드,0,0,7.5,42
3882,20100277,내 남자의 순이,0,0,4.9,92


In [41]:
star_score_df["star_score"] = star_score_df["star_score"].astype(float)
star_score_df["star_user_count"] = star_score_df["star_user_count"].apply(lambda x: np.int(x.replace(",","")))
star_score_df["netizen_score"] = star_score_df["netizen_score"].astype(float)
star_score_df["netizen_count"] = star_score_df["netizen_count"].apply(lambda x: np.int(x.replace(",","")))

<ipython-input-41-ab5b89525caf>:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  star_score_df["star_user_count"] = star_score_df["star_user_count"].apply(lambda x: np.int(x.replace(",","")))
<ipython-input-41-ab5b89525caf>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the r

In [42]:
star_score_df.dtypes

movieCd             object
movieNm             object
star_score         float64
star_user_count      int64
netizen_score      float64
netizen_count        int64
dtype: object

In [43]:
star_score_df

,movieCd,movieNm,star_score,star_user_count,netizen_score,netizen_count
0,20162869,택시운전사,90.02,18929,9.0,49617
1,20150976,신과함께-죄와 벌,80.07,26302,7.8,57870
2,20151228,공조,80.08,14090,8.2,25212
3,20167303,스파이더맨: 홈 커밍,80.08,9933,8.3,18792
4,20172742,범죄도시,90.02,11928,9.1,29016
...,...,...,...,...,...,...
3879,20101283,나루토 질풍전 극장판 4 - 더 로스트 타워,0.00,0,8.1,216
3880,20100775,소라닌,0.00,0,8.4,335
3881,20100011,춤추는 꿈틀이 밴드,0.00,0,7.5,42
3882,20100277,내 남자의 순이,0.00,0,4.9,92


In [52]:
star_score_df.to_csv("일단_star_score.csv",encoding="utf-8")

In [23]:
star_score_df = pd. read_csv("일단_star_score.csv", index_col=0)

movieCd            0
movieNm            0
star_score         0
star_user_count    0
netizen_score      0
netizen_count      0
dtype: int64

In [ ]:
star_score_df.isna().sum()

In [ ]:
star_score_df["star_score"].fillna(0, inplace=True)

In [28]:
star_score_df = star_score_df.reset_index()

In [31]:
star_score_df.drop(['index'],axis=1,inplace=True)

In [53]:
star_score_df

,movieCd,movieNm,star_score,star_user_count,netizen_score,netizen_count
0,20162869,택시운전사,9.2,"18,929",9.0,"49,617"
1,20150976,신과함께-죄와 벌,8.7,"26,302",7.8,"57,870"
2,20151228,공조,8.8,"14,090",8.2,"25,212"
3,20167303,스파이더맨: 홈 커밍,8.8,"9,933",8.3,"18,792"
4,20172742,범죄도시,9.2,"11,928",9.1,"29,016"
...,...,...,...,...,...,...
3879,20101283,나루토 질풍전 극장판 4 - 더 로스트 타워,0.0,0,8.1,216
3880,20100775,소라닌,0.0,0,8.4,335
3881,20100011,춤추는 꿈틀이 밴드,0.0,0,7.5,42
3882,20100277,내 남자의 순이,0.0,0,4.9,92


In [54]:
result2 = pd.read_csv('일단_result2.csv', index_col=0)

In [55]:
data_set = pd.merge(result2,star_score_df,on=['movieCd','movieNm'],how='inner')

In [56]:
data_set

,movieNm,openDt,total_money,audience,screen_num,show_num,배급사,movieCd,director,prdtYear,...,showTm,watchGradeNm,actor_1,actor_2,actor_3,companyNm,star_score,star_user_count,netizen_score,netizen_count
0,택시운전사,20170802.0,95853645649,12186327,1906,184075,(주)쇼박스,20162869,장훈,2017.0,...,137.0,15세이상관람가,이수민,송강호,토마스 크레취만,더램프(주),9.2,"18,929",9.0,"49,617"
1,신과함께-죄와 벌,20171220.0,68350496432,8539495,1912,91301,롯데쇼핑㈜롯데엔터테인먼트,20150976,김용화,2017.0,...,139.0,12세이상관람가,하정우,차태현,주지훈,리얼라이즈픽쳐스(주),8.7,"26,302",7.8,"57,870"
2,공조,20170118.0,63783051326,7817618,1392,142989,(주)씨제이이엔엠,20151228,김성훈,2016.0,...,125.0,15세이상관람가,이재남,현빈,유해진,(주)제이케이필름,8.8,"14,090",8.2,"25,212"
3,스파이더맨: 홈 커밍,20170705.0,59125813820,7258678,1965,164448,소니픽쳐스엔터테인먼트코리아주식회사극장배급지점,20167303,존 왓츠,2017.0,...,133.0,12세이상관람가,제이콥 배덜런,톰 홀랜드,로버트 다우니 주니어,소니픽쳐스엔터테인먼트코리아주식회사극장배급지점,8.8,"9,933",8.3,"18,792"
4,범죄도시,20171003.0,56318014149,6879841,1315,149334,"메가박스중앙(주)플러스엠,(주)키위미디어그룹",20172742,강윤성,2017.0,...,121.0,청소년관람불가,마동석,윤계상,조재윤,(주)홍필름,9.2,"11,928",9.1,"29,016"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3879,나루토 질풍전 극장판 4 - 더 로스트 타워,20101202.0,75017000,11116,58.0,985.0,애니박스 엔터테인먼트,20101283,무라타 마사히코,2010.0,...,85.0,전체관람가,이노우에 카즈히코,모리카와 토시유키,나카무라 치에,애니박스 엔터테인먼트,0.0,0,8.1,216
3880,소라닌,20100826.0,81223000,10913,13.0,578.0,(주)스폰지이엔티,20100775,미키 타카히로,2010.0,...,125.0,12세이상관람가,코라 켄고,미야자키 아오이,NaN,아스믹에이스엔터테인먼트,0.0,0,8.4,335
3881,춤추는 꿈틀이 밴드,20100204.0,72851000,10885,47.0,718.0,에이원엔터테인먼트,20100011,토마스 보르히 닐슨,2008.0,...,80.0,전체관람가,엄상현,이장원,조현정,디스코 오르메네,0.0,0,7.5,42
3882,내 남자의 순이,20100527.0,73108500,10449,99.0,2013.0,(주)새인컴퍼니,20100277,김호준,2010.0,...,91.0,15세이상관람가,박해미,이태성,신이,여해 엔터테인먼트,0.0,0,4.9,92


In [30]:
data_set.to_csv("일단_data_set.csv")  #info랑 detail 합친것

In [7]:
data_set = pd.read_csv('일단_data_set.csv', index_col=0)

In [57]:
data_set.to_excel("일단_data_set.xlsx", index = False)